In [1]:
%load_ext autoreload
%autoreload 2

import LMRt
import GraphEM
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as  plt

In [28]:
dirpath = '/Users/fzhu/Github/GraphEM/docsrc/tutorial/testcases/PPE/data/obs/'
filename = 'tas_sfc_Amon_iCESM_past1000_085001-184912.nc'
model_dirpath = '/Users/fzhu/SynologyDrive/Academic/Projects/volcLMR/LMR_input/data/model/icesm_last_millennium'

with xr.open_dataset(os.path.join(model_dirpath, filename)) as ds:
#     print(ds)
#     print(ds['tas'])
    print(ds['time'])

<xarray.DataArray 'time' (time: 12000)>
array([cftime.DatetimeNoLeap(850, 1, 17, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(850, 2, 14, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(850, 3, 17, 0, 0, 0, 0), ...,
       cftime.DatetimeNoLeap(1849, 10, 17, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1849, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1849, 12, 17, 0, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 0850-01-17 00:00:00 ... 1849-12-17 00:00:00
Attributes:
    description:    time
    long_name:      time
    standard_name:  time


In [30]:
ds = LMRt.Dataset()
ds.load_nc(
    {'tas': os.path.join(model_dirpath, filename)},
    varname_dict={'tas': 'tas'},
    inplace=True
)

In [31]:
ds.seasonalize(list(range(1, 13)), inplace=True)

In [32]:
print(ds)

Dataset Overview
-----------------------

     Name:  tas
   Source:  /Users/fzhu/SynologyDrive/Academic/Projects/volcLMR/LMR_input/data/model/icesm_last_millennium/tas_sfc_Amon_iCESM_past1000_085001-184912.nc
    Shape:  time:1000, lat:96, lon:144


In [33]:
tas = ds.fields['tas']
tas_sub = tas.value[:, 45:55, 65:75]
lat_sub = tas.lat[45:55]
lon_sub = tas.lon[65:75]
year = tas.time
print(np.shape(tas_sub))
# print(year)

(1000, 10, 10)


In [53]:
import cftime

output_dict = {}
output_dict['tas'] = (('time', 'lat', 'lon'), tas_sub)

ds = xr.Dataset(
    data_vars=output_dict,
    coords={
        'time': [cftime.DatetimeNoLeap(y, 1, 1) for y in year],
        'lat': lat_sub,
        'lon': lon_sub,
    }
)
ds.to_netcdf('./testcases/PPE/data/obs/iCESM_subset.nc')

In [50]:
print(ds)

<xarray.Dataset>
Dimensions:  (lat: 10, lon: 10, time: 1000)
Coordinates:
  * time     (time) object 0850-01-01 00:00:00 ... 1849-01-01 00:00:00
  * lat      (lat) float32 -4.737 -2.842 -0.9474 0.9474 ... 8.526 10.42 12.32
  * lon      (lon) float32 162.5 165.0 167.5 170.0 ... 177.5 180.0 182.5 185.0
Data variables:
    tas      (time, lat, lon) float64 300.0 300.0 299.9 ... 299.3 299.3 299.2


In [7]:
# gen pseudoproxy

SNR = 1

pp = np.copy(tas_sub)
nt, nlat, nlon = np.shape(pp)

k = 0
for i in range(nlat):
    for j in range(nlon):
        tas_std = np.std(tas_sub[:,i,j])
        noise_std = tas_std / SNR
        
        np.random.seed(k)
        pp[:,i,j] += np.random.normal(0, noise_std, size=nt)
        
        k += 1

In [8]:
df = pd.read_pickle('./testcases/PAGES2k_HadCRUT/data/proxy/pages2k_dataset.pkl')
df.columns

Index(['paleoData_pages2kID', 'dataSetName', 'archiveType', 'geo_meanElev',
       'geo_meanLat', 'geo_meanLon', 'year', 'yearUnits',
       'paleoData_variableName', 'paleoData_units', 'paleoData_values',
       'paleoData_proxy'],
      dtype='object')

In [9]:
df['paleoData_proxy']

0                  TRW
1             historic
2                  MXD
3                  TRW
4                  TRW
            ...       
687                TRW
688                TRW
689    varve thickness
690               d18O
691              Sr/Ca
Name: paleoData_proxy, Length: 692, dtype: object

In [10]:
df_pp = pd.DataFrame(columns=df.columns)
k = 0
for i in range(nlat):
    for j in range(nlon):
        df_pp.loc[k, 'paleoData_pages2kID'] = f'pp_{k:03d}'
        df_pp.loc[k, 'dataSetName'] = 'pseudoproxy'
        df_pp.loc[k, 'archiveType'] = 'coral'
        df_pp.loc[k, 'geo_meanElev'] = np.nan
        df_pp.loc[k, 'geo_meanLat'] = lat_sub[i]
        df_pp.loc[k, 'geo_meanLon'] = lon_sub[j]
        df_pp.loc[k, 'year'] = year
        df_pp.loc[k, 'yearUnits'] = 'CE'
        df_pp.loc[k, 'paleoData_proxy'] = 'd18O'
        df_pp.loc[k, 'paleoData_values'] = pp[:,i,j]
        k += 1

In [11]:
df_pp.to_pickle('./testcases/PPE/data/proxy/pseudoproxy_dataset.pkl')